In [1]:
from sklearn import svm
from time import time
from pandas import DataFrame
from pathlib import Path
import numpy as np
from sklearn.metrics import f1_score as f1_score_rep
from sklearn.metrics import accuracy_score

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [2]:
print("done")

done


In [3]:
x_test = np.load('/home/jovyan/UNSW/x_test.npy',allow_pickle=True)
x_train = np.load('/home/jovyan/UNSW/x_train.npy',allow_pickle=True)
x_val = np.load('/home/jovyan/UNSW/x_val.npy',allow_pickle=True)
y_test = np.load('/home/jovyan/UNSW/y_test.npy',allow_pickle=True)
y_train = np.load('/home/jovyan/UNSW/y_train.npy',allow_pickle=True)
y_val = np.load('/home/jovyan/UNSW/y_val.npy',allow_pickle=True)

In [4]:
from sklearn import svm
from time import time
from pandas import DataFrame
from pathlib import Path
import numpy as np
from sklearn.metrics import f1_score as f1_score_rep
from sklearn.metrics import accuracy_score

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# make sure that result directory exists before running any of the functions
# result_dir = Path.cwd() / '/content/drive/MyDrive/Cybersecurity/Datasets/results'
# if not result_dir.exists():
#     result_dir.mkdir()


def tune_with_halving_grid_search(x_train, y_train, param_grid):
    svc = svm.SVC()

    start = time()
    halving_gs_results = HalvingGridSearchCV(
        svc,
        param_grid,
        cv=5,
        factor=3,
        min_resources='exhaust',
        scoring='f1_weighted'
    ).fit(x_train, y_train)

    duration = time() - start

    results = DataFrame(halving_gs_results.cv_results_)
    results.loc[:, 'mean_test_score'] *= 100
    # results.to_csv(result_dir / 'halving_svc_results.csv')

    # take the most relevant columns and sort (for readability). Remember to sort on the iter columns first, so we see
    # the models with the most training data behind them first.
    results = results.loc[:, ('iter', 'rank_test_score', 'mean_test_score', 'params')]
    results.sort_values(by=['iter', 'rank_test_score'], ascending=[False, True], inplace=True)

    return results, duration

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# from hyperparam_tuning import tune_with_grid_search, tune_with_halving_grid_search


svc_params = {
    'C': [0.1, 0.5, 1,5,10],
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
    'tol': [1e-3, 1e-2]
}

# gs_results, gs_duration = tune_with_grid_search(x_train, y_train, svc_params)
halving_results, halving_duration = tune_with_halving_grid_search(x_val, y_val, svc_params)

print(halving_results.head())

score2 = halving_results['mean_test_score'].iloc[0]
params2 = halving_results['params'].iloc[0]



best_svm = svm.SVC(**params2)
best_svm.fit(x_train, y_train)
# accuracy2 = accuracy_score(y_test, svc2.predict(x_test))

print(f'Best score for HalvingGridSearchCv is {score2:.3f}, took {halving_duration:.2f} seconds')
print(f'Params: {params2}')
print(f'Corresponding test accuracy: {accuracy2 * 100:.2f}%')
print("Micro F1 Score: ", f1_score_rep(y_test, best_svm.predict(x_test), average="micro"))
print("Macro F1 Score: ", f1_score_rep(y_test, best_svm.predict(x_test), average="macro"))


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.datasets import load_digits

# best_svm = svm.SVC(C=10, kernel='poly', tol=0.01)
# best_svm.fit(x_train, y_train)

y_pred = best_svm.predict(x_test)

conf_matrix = confusion_matrix(y_test, y_pred)
# tn, fp, fn, tp = conf_matrix.ravel()
cm = conf_matrix
fp = cm.sum(axis=0) - np.diag(cm)
fn = cm.sum(axis=1) - np.diag(cm)
tp = np.diag(cm)
tn = cm.sum() - (fp + fn + tp)


fpr = fp / (fp + tn)
tpr = tp / (tp + fn)
fnr = fn / (fn + tp)
tnr = tn / (tn + fp)
precision = precision_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
# auc = roc_auc_score(y_test, best_svm.predict_proba(x_test), multi_class='ovr')

print(f"FPR: {fpr}")
print(f"TPR: {tpr}")
print(f"FNR: {fnr}")
print(f"TNR: {tnr}")
print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")
print(f"F1-score: {f1}")
# print(f"AUC: {auc}")

print(f"FPR: {np.mean(fpr)}")
print(f"tpr: {np.mean(tpr)}")
print(f"fnr: {np.mean(fnr)}")
print(f"tnr: {np.mean(tnr)}")

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels=best_svm.classes_)
disp.plot()
plt.show()

import pandas as pd
metrics = {
    "model" : "svm",
    "Accuracy": accuracy,
    "Precision": precision,
    "F1-score": f1,
    "FPR": np.mean(fpr),
    "TPR": np.mean(tpr),
    "FNR": np.mean(fnr),
    "TNR": np.mean(tnr),
    "AUC": auc
}
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv("/home/jovyan/UNSW/model.csv", mode='a',header = False, index=False)

In [ ]:
# print(f"FPR: {fpr}")
# print(f"TPR: {tpr}")
# print(f"FNR: {fnr}")
# print(f"TNR: {tnr}")
# print(f"Precision: {precision}")
# print(f"Accuracy: {accuracy}")
# print(f"F1-score: {f1}")

In [ ]:
# from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# auc = roc_auc_score(y_test, best_svm.predict(x_test), multi_class='ovr')
# print(f"AUC: {auc}")

In [ ]:
# print(f"FPR: {np.mean(fpr)}")
# print(f"tpr: {np.mean(tpr)}")
# print(f"fnr: {np.mean(fnr)}")
# print(f"tnr: {np.mean(tnr)}")

# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels=best_svm.classes_)
# disp.plot()
# plt.show()